In [ ]:
import glob
from os.path import dirname, join

import numpy as np
from scipy.interpolate import interp1d
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from winsound import Beep

In [ ]:
def decompose_transformation_matrix(transformation):
    translation = transformation[0:3, 3]
    rotation = transformation[0:3, 0:3]
    return translation, rotation

In [ ]:

start_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformation_matrices_2\\1667908110_100000000_transformation.txt"
end_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformation_matrices_2\\1667908113_100000000_transformation.txt"
stm = np.loadtxt(start_path)
strm, srm = decompose_transformation_matrix(stm)

etm = np.loadtxt(end_path)
etrm, erm = decompose_transformation_matrix(etm)

r0 = R.from_euler("ZYX", [90, -30, 0], degrees=True)
sr = R.from_matrix([srm,erm])
sr = R.concatenate([sr,r0])
# er = R.from_quat()

key_rots = sr

key_times = [0,0.5, 1]

In [ ]:
slerp = Slerp(key_times, key_rots)

times = np.arange(0, 1, 0.1)
interp_rots = slerp(times)

In [ ]:
interp_rots.as_euler('xyz', degrees=True)

In [ ]:
sr.as_euler('xyz', degrees=True)

In [ ]:
def plot_rotated_axes(ax, r, name=None, offset=(0, 0, 0), scale=1):
    colors = ("#FF6666", "#005533", "#1199EE")  # Colorblind-safe RGB
    loc = np.array([offset, offset])
    for i, (axis, c) in enumerate(zip((ax.xaxis, ax.yaxis, ax.zaxis), colors)):
        axlabel = axis.axis_name
        axis.set_label_text(axlabel)
        axis.label.set_color(c)
        axis.line.set_color(c)
        axis.set_tick_params(colors=c)
        line = np.zeros((2, 3))
        line[1, i] = scale
        line_rot = r.apply(line)
        line_plot = line_rot + loc
        ax.plot(line_plot[:, 0], line_plot[:, 1], line_plot[:, 2], c)
        text_loc = line[1]*1.2
        text_loc_rot = r.apply(text_loc)
        text_plot = text_loc_rot + loc[0]
        ax.text(*text_plot, axlabel.upper(), color=c, va="center", ha="center")
    ax.text(*offset, name, color="k", va="center", ha="center", bbox={"fc": "w", "alpha": 0.8, "boxstyle": "circle"})

In [ ]:
r0 = R.identity()
r1 = R.from_euler("ZYX", [90, -30, 0], degrees=True)  # intrinsic
r2 = R.from_euler("zyx", [90, -30, 0], degrees=True)  # extrinsic

ax = plt.figure().add_subplot(projection="3d", proj_type="ortho")
plot_rotated_axes(ax, r0, name="r0", offset=(0, 0, 0))
plot_rotated_axes(ax, r1, name="r1", offset=(3, 0, 0))
plot_rotated_axes(ax, r2, name="r2", offset=(6, 0, 0))
_ = ax.annotate(
    "r0: Identity Rotation\n"
    "r1: Intrinsic Euler Rotation (ZYX)\n" 
    "r2: Extrinsic Euler Rotation (zyx)", xy=(0.6, 0.7), xycoords="axes fraction", ha="left" )
ax.set(xlim=(-1.25, 7.25), ylim=(-1.25, 1.25), zlim=(-1.25, 1.25))
ax.set(xticks=range(-1, 8), yticks=[-1, 0, 1], zticks=[-1, 0, 1])
ax.set_aspect("equal", adjustable="box")
ax.figure.set_size_inches(6, 5)
plt.tight_layout()

In [ ]:
ax = plt.figure().add_subplot(projection="3d", proj_type="ortho")
for idx, rot in enumerate(interp_rots):
    plot_rotated_axes(ax, rot, name="r0", offset=(idx*3, 0, 0))

ax.set_aspect("equal", adjustable="box")
ax.figure.set_size_inches(30, 50)
plt.tight_layout()

In [ ]:
key_tsl = np.vstack([strm, etrm])
key_times = [0, 1]
linfit = interp1d(key_times, key_tsl, axis=0)


arr = linfit(np.arange(0,1,0.1))

In [ ]:
arr

In [ ]:
arr[:,2]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(arr[:,0],arr[:,1],arr[:,2])


In [ ]:
arr[0]

In [ ]:
x = np.identity(4)
x[:,3] = arr[0]
x

In [ ]:
rot_matrix = r1.as_matrix()



In [ ]:
def compose_transformation_matrix(rotation_matrix, translation_matrix):
    exp_mat = np.append(rotation_matrix, np.expand_dims(translation_matrix, axis = 1), axis=1)
    return np.append(exp_mat, np.array([[0,0,0,1]]), axis=0)

In [ ]:
compose_transformation_matrix(rot_matrix, arr[0])

In [ ]:
src_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformation_matrices_2\\"
frames_to_interpolate = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\converted\\pcd\\"
target_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\interpolation\\"
stm = np.loadtxt(start_path)
strm, srm = decompose_transformation_matrix(stm)

filenames_to_interpolate = sorted(glob.glob(frames_to_interpolate + "166*.pcd"))
filenames_fixed_points = sorted(glob.glob(src_folder + "*.txt"))

In [ ]:
print(get_timestamps(filenames_fixed_points))
print(get_timestamps(filenames_to_interpolate))
print(len(get_timestamps(filenames_to_interpolate)))

In [ ]:
def get_timestamps(filenames):
    timestamps = []
    for filename in filenames:
        x = filename.split("\\")[-1]
        y = x.split("_")
        timestamps.append(int(y[0]+y[1]))
    return timestamps


fixed_rotations = []
fixed_translations = []
for f in filenames_fixed_points:
    stm = np.loadtxt(f)
    strm, srm = decompose_transformation_matrix(stm)
    fixed_rotations.append(srm)
    fixed_translations.append(strm)

# print(fixed_rotations)

key_rots = R.from_matrix(fixed_rotations)
key_times = get_timestamps(filenames_fixed_points)
print(key_times)
slerp = Slerp(key_times, key_rots)

times = get_timestamps(filenames_to_interpolate)
interp_rots = slerp(times)

key_tsl = np.vstack(fixed_translations)
linfit = interp1d(key_times, key_tsl, axis=0)
interp_trans = linfit(times)

trajectories = []
for idx, time in enumerate(times):
    x, y, z = interp_trans[idx]
    qx, qy, qz, qw = interp_rots[idx].as_quat()
    trajectories.append(f"{time/1000000000} {x} {y} {z} {qx} {qy} {qz} {qw}")

with open(target_folder + "interpolated_poses_tum.txt", 'w') as fp:
    fp.write('\n'.join(trajectories))

In [ ]:
trajectories[0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for x,y,z in interp_trans:
    ax.scatter(x,y,z)


In [ ]:
ax = plt.figure().add_subplot(projection="3d", proj_type="ortho")
for idx, rot in enumerate(interp_rots):
    plot_rotated_axes(ax, rot, name="", offset=(idx*3, 0, 0))

# ax.set_aspect(adjustable="box")
ax.figure.set_size_inches(30, 50)
plt.tight_layout()

In [ ]:
arr = interp_trans
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(arr[:,0],arr[:,1],arr[:,2])

In [ ]:
interp_rots[1].as_quat()

In [ ]:
x, y, z = interp_trans[0]

In [ ]:
x

#Convert kiss-icp tum poses to tum icp poses with corrected timestamps

In [ ]:
input_tum_file = r"C:\Users\amoff\results\2023-08-10_20-24-25\pcd_poses_tum.txt"
output_tum_file = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\rotated_and_translated\100000000\interpolation\2023-08-10_20-24-25_pcd_poses_tum.txt"
offset = 1667908110
contains_drone_data_as_first_entry = True

In [ ]:
with open(input_tum_file) as f:
    lines = f.readlines()

if contains_drone_data_as_first_entry:
    lines = lines[1:]

return_lines = []
for l in lines:
    entries = l.split(" ")
    index = float(entries[0]) - (1 if contains_drone_data_as_first_entry else 0)
    entries[0] = str(index/10 + offset)
    print(entries[0])
    return_lines.append(" ".join(entries))

print(return_lines)

with open(output_tum_file, 'w') as fp:
    fp.write(''.join(return_lines))
    print(f"Wrote tum poses file to: {output_tum_file}")